In [39]:
from bs4 import BeautifulSoup
import requests

In [40]:

headers={"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"}
target_url = "https://www.booking.com/hotel/us/the-lenox.html?checkin=2022-12-28&checkout=2022-12-29&group_adults=2&group_children=0&no_rooms=1&selected_currency=USD"
resp = requests.get(target_url, headers=headers)
print(resp.status_code)

200


In [56]:
def scrape(url):
    l=list()
    g=list()
    o={}
    k={}
    fac=[]
    fac_arr=[]

    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    o["name"]=soup.find("h2",{"class":"pp-header__title"}).text
    o["address"]=soup.find("span",{"class":"hp_address_subtitle"}).text.strip("\n")
    outer_div=soup.find("div",{"class":"a3b8729ab1 d86cee9b25"})
    contents = outer_div.contents
    o["rating"] = contents[0].strip()
    o["stars"] = len(soup.find("span",{"class":"a455730030 d542f184f1"}).contents)
    

    fac=soup.find_all("span",{"class":"a5a5a75131"})
    for i in range(0,len(fac)):
        fac_arr.append(fac[i].text.strip("\n"))
    ids= list()
    targetId=list()
    try:
        tr = soup.find_all("tr")
    except:
        tr = None
    for y in range(0,len(tr)):
        try:
            id = tr[y].get('data-block-id')
        except:
            id = None
        if( id is not None):
            ids.append(id)
    print("ids are ",len(ids))
    for i in range(0,len(ids)):
        try:
            allData = soup.find("tr",{"data-block-id":ids[i]})
            try:
                rooms = allData.find("span",{"class":"hprt-roomtype-icon-link"})
            except:
                rooms=None
            if(rooms is not None):
                last_room = rooms.text.replace("\n","")
            try:
                k["room"]=rooms.text.replace("\n","")
            except:
                k["room"]=last_room
            price = allData.find("div",{"class":"bui-price-display__value prco-text-nowrap-helper prco-inline-block-maker-helper prco-f-font-heading"})
            k["price"]=price.text.replace("\n","")
            g.append(k)
            k={}
        except:
            k["room"]=None
            k["price"]=None

    location_blocks = soup.find_all('div', class_='dc5041d860 c72df67c95 fb60b9836d')

    # Find all distance blocks
    distance_blocks = soup.find_all('div', class_='a53cbfa6de f45d8e4c32 c875b9e968')

    # Assuming each location block corresponds to a distance block in order
    locations_and_distances = []
    for location, distance in zip(location_blocks, distance_blocks):
        location_name = location.get_text().strip()
        distance_text = distance.get_text().strip()
        locations_and_distances.append((location_name, distance_text))
       

    l.append(g)
    l.append(o)
    l.append(fac_arr)
    l.append(locations_and_distances)
    return l

In [58]:
target_url2 = "https://www.booking.com/hotel/jp/villa-fontaine-grand-tokyo-ariake.html?aid=304142&label=gen173nr-1FCAEoggI46AdIM1gEaJgCiAEBmAEhuAEHyAEM2AEB6AEB-AELiAIBqAIDuALp34ewBsACAdICJGIyMTY0NTZkLTA4ZmYtNGI2Yy1iNjlmLWNjYWVmNDQ3YTBkY9gCBuACAQ&sid=93001eae5bdb2370c073f0a3e1ca7ead&all_sr_blocks=583523559_309180516_2_42_0&checkin=2024-04-12&checkout=2024-04-20&dest_id=-246227&dest_type=city&group_adults=2&group_children=0&hapos=1&highlighted_blocks=583523559_309180516_2_42_0&hpos=1&matching_block_id=583523559_309180516_2_42_0&no_rooms=1&req_adults=2&req_children=0&room1=A%2CA&sb_price_type=total&sr_order=popularity&sr_pri_blocks=583523559_309180516_2_42_0__32524000&srepoch=1711403033&srpvid=b7cb98c74ffa0050&type=total&ucfs=1&#hotelTmpl"
target_url3 = "https://www.booking.com/hotel/jp/karaksa-hotel-grande-shin-osaka-tower.html?aid=304142&label=gen173nr-1FCAQoggJCDHNlYXJjaF9vc2FrYUgxWARomAKIAQGYATG4AQfIAQzYAQHoAQH4AQOIAgGoAgO4Ap6Pl7AGwAIB0gIkNTJjMWY2OWEtZTQ4ZS00YjM5LTg4ZmQtNzQ1MDllNGQ5ODM52AIF4AIB&sid=93001eae5bdb2370c073f0a3e1ca7ead&all_sr_blocks=466033402_387098578_2_0_0;checkin=2024-04-01;checkout=2024-04-09;dist=0;group_adults=2;group_children=0;hapos=1;highlighted_blocks=466033402_387098578_2_0_0;hpos=1;matching_block_id=466033402_387098578_2_0_0;nflt=price%3DUSD-min-max-1;no_rooms=1;req_adults=2;req_children=0;room1=A%2CA;sb_price_type=total;sr_order=popularity;sr_pri_blocks=466033402_387098578_2_0_0__15876000;srepoch=1711657772;srpvid=e9e08b089d2e00ea;type=total;ucfs=1&#hotelTmpl"

l =scrape(target_url3)
print(l)

ids are  4
[[{'room': 'Standard Twin Room - Non-Smoking', 'price': 'US$1,049'}, {'room': 'Standard Twin Room - Non-Smoking', 'price': 'US$1,244'}, {'room': 'Standard Twin Room - Non-Smoking', 'price': 'US$1,146'}], {'name': 'karaksa hotel grande Shin-Osaka Tower', 'address': '532-0003 Osaka Prefecture, Osaka, Yodogawa-ku Miyahara 3-3-24, Japan', 'rating': '8.7', 'stars': 4}, ['Non-smoking rooms', 'Facilities for disabled guests', 'Free WiFi', 'Restaurant', '24-hour front desk', 'Lift', 'Luggage storage', 'Heating', 'Daily housekeeping', 'Very good breakfast', 'Non-smoking rooms', 'Facilities for disabled guests', 'Free WiFi', 'Restaurant', '24-hour front desk', 'Lift', 'Luggage storage', 'Heating', 'Daily housekeeping', 'Very good breakfast', 'Toilet paper', 'Towels', 'Bidet', 'Slippers', 'Private bathroom', 'Toilet', 'Free toiletries', 'Hairdryer', 'Shower', 'Linen', 'Alarm clock', 'Extra long beds (> 2 metres)', 'Electric kettle', 'Refrigerator', 'Socket near the bed', 'Clothes rack'

In [64]:
def scrape_hotels(city, checkin_date, checkout_date, adults = 2, children = 0, min_price = "min", max_price = "max"):
    url = f"https://www.booking.com/searchresults.html?ss={city}&lang=en-us&sb=1&src_elem=sb&src=index&checkin={checkin_date}&checkout={checkout_date}&group_adults={adults}&no_rooms=1&group_children={children}&nflt=price%3DUSD-{min_price}-{max_price}-1"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/107.0.0.0 Safari/537.36"
    }
    print(url)
    resp = requests.get(url, headers=headers)
    soup = BeautifulSoup(resp.text, 'html.parser')
    hotels = soup.find_all('a', class_='a78ca197d0')
    hotel_links = [link['href'] for link in hotels]
    return hotel_links

In [74]:
hotels = scrape_hotels("Paris", "2024-05-01", "2024-05-10", 2, 0, m)


https://www.booking.com/searchresults.html?ss=Paris&lang=en-us&sb=1&src_elem=sb&src=index&checkin=2024-05-01&checkout=2024-05-10&group_adults=2&no_rooms=1&group_children=0&nflt=price%3DUSD-min-max-1


In [75]:
print(hotels[0])

https://www.booking.com/hotel/fr/paris-j-39-adore-amp-spa.html?aid=304142&label=gen173nr-1FCAQoggJCDHNlYXJjaF9wYXJpc0gxWARomAKIAQGYATG4ARfIAQzYAQHoAQH4AQOIAgGoAgO4AvjFrLAGwAIB0gIkYmVlZmU1OTMtMDgxOC00ZWE4LWE2MDctYjIwMmZiNjFmMTdi2AIF4AIB&ucfs=1&arphpl=1&checkin=2024-05-01&checkout=2024-05-10&group_adults=2&req_adults=2&no_rooms=1&group_children=0&req_children=0&hpos=1&hapos=1&sr_order=popularity&nflt=price%3DUSD-min-max-1&srpvid=d57194fcaa17006e&srepoch=1712005882&all_sr_blocks=713048601_298165938_0_2_0&highlighted_blocks=713048601_298165938_0_2_0&matching_block_id=713048601_298165938_0_2_0&sr_pri_blocks=713048601_298165938_0_2_0__496638&from_sustainable_property_sr=1&from=searchresults#hotelTmpl


In [76]:
print(scrape(hotels[0]))

ids are  15
[[{'room': 'Classic - Elegant with Jacuzzi', 'price': 'US$4,738'}, {'room': 'Classic - Elegant with Jacuzzi', 'price': 'US$5,351'}, {'room': 'Classic - Elegant with Jacuzzi', 'price': 'US$5,171'}, {'room': 'Superior - Boudoir with Jacuzzi', 'price': 'US$5,449'}, {'room': 'Superior - Boudoir with Jacuzzi', 'price': 'US$6,062'}, {'room': 'Superior - Boudoir with Jacuzzi', 'price': 'US$5,881'}, {'room': 'Deluxe - Precious with Jacuzzi', 'price': 'US$5,922'}, {'room': 'Deluxe - Precious with Jacuzzi', 'price': 'US$6,535'}, {'room': 'Deluxe - Precious with Jacuzzi', 'price': 'US$6,355'}, {'room': 'Deluxe - Passion with Jacuzzi', 'price': 'US$6,159'}, {'room': 'Deluxe - Passion with Jacuzzi', 'price': 'US$6,772'}, {'room': 'Deluxe - Passion with Jacuzzi', 'price': 'US$6,592'}, {'room': 'Prestige – Sublime with Jacuzzi', 'price': 'US$7,344'}, {'room': 'Prestige – Sublime with Jacuzzi', 'price': 'US$7,957'}, {'room': 'Prestige – Sublime with Jacuzzi', 'price': 'US$7,777'}], {'name'